In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()#'vice')

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [22]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (95.41 seconds): Collecting URLs... rt                  

n_total     26287
n_unique    26217
dtype: int64

,pubs,actual_pub
https://breitbart.com/national-security/2019/10/07/kurdish-forces-who-defeated-islamic-state-call-trump-syria-withdrawal-a-stab-in-the-back,"axios, breitbart",breitbart
https://breitbart.com/politics/2019/10/05/nolte-anti-kavanaugh-book-written-by-new-york-times-reporters-flops,"realclearpolitics, breitbart",breitbart
https://businessinsider.co.za/bruce-whitfield-creating-jobs-21st-century-style-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/bushbuckridge-local-municipality-built-a-road-around-an-electric-po-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/esim-vodacom-mtn-launch-south-africa-what-it-means-2019-10,"businessinsider, news24",businessinsider
https://cbsnews.com/news/warren-stands-by-account-of-being-pushed-out-of-her-first-teaching-job-because-of-pregnancy,"cbs, realclearpolitics",cbs
https://cnbc.com/2019/10/03/vices-400-million-deal-for-refinery29-illustrates-the-pointlessness-of-private-valuations.html,"investing.com, axios",investing.com
https://cnbc.com/2019/10/08/china-state-tv-suspends-nba-broadcasts-after-morey-hong-kong-tweet.html,"cnbc, realclearpolitics",cnbc
https://foxnews.com/politics/fox-news-poll-results-september-29-october-2-2019-wisconsin,"washingtontimes, nationalreview",washingtontimes
https://hotair.com/archives/jazz-shaw/2019/10/06/pilot-chased-ufo-reveals-lot-story,"townhall, hotair",hotair


huffpost                  72
cnn                      801
investing.com            990
politico                 215
time                      21
cnbc                     152
fox                      333
bbc                      275
businessinsider          561
morningstar               95
wsj                      103
nytimes                  106
guardian                 156
reuters                 1153
washingtontimes          454
washingtonpost           132
cbs                      406
marketwatch              142
atlantic                 100
vice                      97
npr                      868
newrepublic               20
yahoo                    393
independent              715
heritage                 287
zdnet                    402
townhall                 606
abcnews                   95
hotair                    84
cbc                      302
                        ... 
metro                    231
msnbc                    104
nationalreview           733
news24        

In [18]:
collect_results[2]['thinkprogress']

82

# 2. 모은 URL중 이미 받은 것은 버린다

In [19]:
select_results = crawler.select(); display(*select_results)

100.00% (2.94 seconds): Selecting URLs... cryptonews          

n_total     727
n_unique    727
dtype: int64

,pubs,actual_pub


huffpost                  2
cnn                      12
investing.com            52
politico                  2
time                      1
cnbc                      3
fox                      13
bbc                       2
businessinsider          33
morningstar              11
wsj                       4
nytimes                  16
guardian                  3
reuters                 101
washingtontimes           7
washingtonpost           10
cbs                       3
marketwatch               7
atlantic                  0
vice                      2
npr                       0
newrepublic               0
yahoo                    54
independent               6
heritage                  0
zdnet                     1
townhall                  0
abcnews                   3
hotair                    0
cbc                       1
                       ... 
metro                    30
msnbc                     3
nationalreview            7
news24                    2
techcrunch          

In [20]:
crawler.urls_selected['thinkprogress']

set()

# 3. 해당 URL의 뉴스를 다운로드한다

In [21]:
download_results = crawler.download(); download_results

100.00% (235.60 seconds): Downloading... euronews            

,downloaded,trashed
abcnews,3,0
afr,4,0
aljazeera,1,17
americanconservative,0,0
arirang,0,0
arstechnica,1,0
arynews,5,2
atlantic,0,0
axios,10,6
bbc,2,0


In [20]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

['d80494a4761ead4276e510dc4f64762e319902e0',
 'f25b1d06e58c05786ad57fe9bea9817859de3861',
 '02d668060004e25c2299c9a23445e38a9c6ec771',
 '2d3e330325b35d2c622806feac055478a4a61559',
 '2e87c621d1ffb7f28485496eb82814891ff5dd1b',
 '40cf0a5a2b87e040cd211da1855cb615702cb0ca',
 'feefa98b143ce76ef3cadf0a2764ca634f773fee',
 '0a900188f145b48aad8c70bde741aea563dba542',
 'ef7e33fa30c8be4b6d90681b383234ee4f73e51a',
 '64cc9a5e5a51520306154fda70d482b57a7420b0',
 '12e43a97340dc5a9f9270b38a9705e461456041b',
 'd71b4a23216b50deb276d3ccd8c1905939f5a9c2',
 '9a5e5f813637acd5359da986e562fa9ad9858f4b',
 '44c36bff1fe4d64fb468131f1221959fa30718d6',
 'dbf61f3dd462f9c650d0b2f42ca69717701fc7d1',
 '385dbc7ad54410ac2078055044e34c6e30cfde4b',
 'a31fe291585e194540805022e45df9ebdbabc9fc',
 '66afae226b2dc5d329fc731f7931f9f6d6f37e23']